In [86]:
from utils.load_dataset import load_dataset
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.evaluation import rmse_score, mrr_score, precision_recall_score
import numpy as np

In [6]:
ds_df = load_dataset()

In [7]:
ds_df['ratings']

,UserID,MovieID,Rating,Timestamp
0,1,185,5.0,838983525
1,1,231,5.0,838983392
2,1,292,5.0,838983421
3,1,316,5.0,838983392
4,1,329,5.0,838983392
...,...,...,...,...
10000048,71567,2107,1.0,912580553
10000049,71567,2126,2.0,912649143
10000050,71567,2294,5.0,912577968
10000051,71567,2338,2.0,912578016


In [20]:
interactions = Interactions(
    user_ids=ds_df['ratings']['UserID'].to_numpy(dtype=np.int32),
    item_ids=ds_df['ratings']['MovieID'].to_numpy(dtype=np.int32),
    ratings=ds_df['ratings']['Rating'].to_numpy(dtype=np.float32),
    timestamps=ds_df['ratings']['Timestamp'].to_numpy(dtype=np.int32)
)

In [63]:
train, test = random_train_test_split(interactions, test_percentage=0.005)

In [65]:
len(test)

50001

In [66]:
print(test)

<Interactions dataset (71568 users x 65134 items x 50001 interactions)>


## ExplicitFactorizationModel

In [68]:
ef_model = ExplicitFactorizationModel(n_iter=3, use_cuda=True)

In [69]:
ef_model.fit(train, verbose=True)

Epoch 0: loss 1.3316299344570264
Epoch 1: loss 1.26178592517995
Epoch 2: loss 1.2779347658988514


In [74]:
ef_rmse = rmse_score(ef_model, test)

In [75]:
ef_mrr = mrr_score(ef_model, test)

In [76]:
ef_pr = precision_recall_score(ef_model, test, k=10)

In [77]:
print(ef_rmse)

1.1371931


In [78]:
print(ef_mrr)

[0.00022008 0.00014901 0.00034165 ... 0.00026603 0.00024172 0.00031656]


In [82]:
print(np.mean(ef_mrr), np.std(ef_mrr))

0.000270617531924396 0.0004562936652547344


In [83]:
print(np.mean(ef_pr[0]), np.std(ef_pr[0]))

1.1313923668728317e-05 0.0010636091208729127


In [84]:
print(np.mean(ef_pr[1]), np.std(ef_pr[1]))

1.1074475548755227e-05 0.0011645921777011343


## ImplicitFactorizationModel

In [92]:
if_model = ImplicitFactorizationModel(n_iter=3, use_cuda=True)

In [93]:
if_model.fit(train, verbose=True)

Epoch 0: loss 0.11709153154527342
Epoch 1: loss 0.0890135140154594
Epoch 2: loss 0.08240981313609008


In [94]:
if_rmse = rmse_score(if_model, test)

In [95]:
if_mrr = mrr_score(if_model, test)

In [96]:
if_pr = precision_recall_score(if_model, test, k=10)

In [97]:
print(if_rmse)

39.67255


In [98]:
print(if_mrr)

[0.0038616  0.00302115 0.001287   ... 0.00106724 0.00098522 0.05263158]


In [99]:
print(np.mean(if_mrr), np.std(if_mrr))

0.005678321749415576 0.028690135715332594


In [100]:
print(np.mean(if_pr[0]), np.std(if_pr[0]))

0.0011728767536581686 0.01080121738807531


In [101]:
print(np.mean(if_pr[1]), np.std(if_pr[1]))

0.007413450524685251 0.07704897056826505
